In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import cv2

#from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Conv2D,Flatten,MaxPooling2D,AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.applications.vgg16 import VGG16


# import keras_tuner as kt


2024-10-19 21:23:17.249865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 21:23:17.253508: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 21:23:17.353879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 21:23:17.408236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 21:23:17.504006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
folder_path="/home/aman/Desktop/Python/ML/Datasets/data/"

In [3]:
img_path = []

In [4]:
for file in os.listdir(folder_path+"with_mask"):
  img_path.append(file)


In [5]:
df=pd.DataFrame({'image':img_path,'mask':1})

In [6]:
df.head()

,image,mask
0,with_mask_1.jpg,1
1,with_mask_10.jpg,1
2,with_mask_100.jpg,1
3,with_mask_1000.jpg,1
4,with_mask_1001.jpg,1


In [7]:
new_img= []

In [8]:
for file in os.listdir(folder_path+"without_mask"):
  new_img.append(file)
new_df=pd.DataFrame({'image':new_img,'mask':0})

In [9]:
df.tail()

,image,mask
3720,with_mask_995.jpg,1
3721,with_mask_996.jpg,1
3722,with_mask_997.jpg,1
3723,with_mask_998.jpg,1
3724,with_mask_999.jpg,1


In [10]:
new_df.head()

,image,mask
0,without_mask_1.jpg,0
1,without_mask_10.jpg,0
2,without_mask_100.jpg,0
3,without_mask_1000.jpg,0
4,without_mask_1001.jpg,0


In [11]:
result = pd.concat([df,new_df],ignore_index=True)

In [12]:
result['mask'].value_counts()

mask
0    3828
1    3725
Name: count, dtype: int64

In [13]:
final_df= result.sample(frac=1, random_state=1).reset_index(drop=True)

In [72]:
final_df['mask'][final_df['mask']==0]='0'

In [73]:
train_df = final_df.sample(frac=1,random_state=0).iloc[:6553]
test_df = final_df.sample(frac=1,random_state=0).iloc[6553:]
path = "/home/aman/Desktop/Python/ML/Datasets/data/all/"
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=path,
                                                    x_col='image',
                                                    y_col='mask',
                                                    target_size=(224,224),
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=path,
                                                    x_col='image',
                                                    y_col='mask',
                                                    target_size=(224,224),
                                                  class_mode='binary')



Found 6553 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [14]:
data =[]
label = []

In [15]:
for i in range(final_df.shape[0]):
  name,mask= (final_df.iloc[i,:]).tolist()
  if (mask == 1):
    img_path=os.path.join(folder_path+"with_mask/",name)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    img=np.array(img)
    label.append(1)

    data.append(img)
  else :
    img_path = os.path.join(folder_path+'without_mask/',name)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    img=np.array(img)
    label.append(0)

    data.append(img)



In [16]:
X=np.array(data)
y=np.array(label)
X=X/255

In [17]:
X.shape

(7553, 224, 224, 3)

In [18]:
y.shape

(7553,)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
X_train.shape

(6042, 150, 150, 3)

In [22]:
X_test.shape

(1511, 150, 150, 3)

In [74]:
conv_base= ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
conv_base.trainable= True
for layers in conv_base.layers:
  if ('conv5_block3' in layers.name):
    layers.trainable=True
  else:
    layers.trainable=False



model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_regularizer=tensorflow.keras.regularizers.l2(0.05)))
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
conv_base = VGG16()

model = Sequential()
for layer in conv_base.layers[:-1]:
    layer.trainable = False
    model.add(layer)



In [24]:

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 134,260,544 (512.16 MB)

In [25]:
model.add(Dense(1,activation='sigmoid'))

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [27]:
model.compile(optimizer='Adam',metrics=['accuracy'],loss='binary_crossentropy')

In [28]:
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test),batch_size=32)

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "fc1" is incompatible with the layer: expected axis -1 of input shape to have value 25088, but received input with shape (None, 8192)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 150, 150, 3), dtype=float32)
  • training=True
  • mask=None

In [28]:
model.save("face_mask.keras")

In [ ]:
cap= cv2.VideoCapture(0)

while True:
    ret ,frame = cap.read()
    # call the detection 
    frame =cv2.resize(frame,(150,150))
    y_pred= detectface_mask(frame)
    cv2.imshow("window",frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()

def detectface_mask(img):
    y_pred = model.predict(img.reshape(1,150,150,3))
    return y_pred[0][0]

sample_1=cv2.imread('/home/aman/Desktop/Python/ML/Mask/mask.png')
sample_1 = cv2.resize(sample_1,(150,150))

detectface_mask(sample_1)

In [41]:
final_df

,image,mask
0,with_mask_3172.jpg,1
1,with_mask_2976.jpg,1
2,without_mask_28.jpg,0
3,with_mask_2166.jpg,1
4,without_mask_372.jpg,0
...,...,...
7548,with_mask_1813.jpg,1
7549,without_mask_2319.jpg,0
7550,without_mask_1228.jpg,0
7551,with_mask_121.jpg,1
